In [1]:
import torch
import monai
import dataloader
import os
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import MSELoss
from torch.nn import BCEWithLogitsLoss
import numpy as np
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import torch.optim.lr_scheduler
from monai.losses import DiceLoss
from monai.losses import FocalLoss
from monai.networks.nets import UNet
import torchmetrics
from torchmetrics.classification import BinaryPrecision, BinaryRecall
import math
import torch.nn.init as init

In [2]:
data_path = '../data/hm30rad/'
df = pd.read_csv('../data/train_labels.csv')
names = df['tomo_id'].astype(str).unique().tolist()

In [ ]:
import optuna
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from monai.losses import DiceLoss
from monai.networks.nets import UNet
from optuna.pruners import MedianPruner
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the objective function for Optuna
def objective(trial):
    # ------------------------------ #
    #        HYPERPARAMETERS         #
    # ------------------------------ #
    # lr = trial.suggest_float("lr", 5e-6, 5e-5, log=True)
    lr = 5e-4
    zero_threshold = trial.suggest_float("zero_threshold", 0.4, 0.75)
    
    # theta = trial.suggest_float("theta", 0.1, 0.9)
    # warmup_epochs = trial.suggest_int("warmup_epochs", 8, 12)
    warmup_epochs=10
    pos_weight_val = 100
    gamma = 1.5
    decay = 0.9
    # reg_str=1e-3
    dropout=0.3
    num_epochs = 35
    batch_size = 32
    # pos_weight = torch.tensor([pos_weight_val], device=device)  # Replace 'value' with your desired pos_weight
    
    ''' Unused Hyperparameters
    dropout = trial.suggest_float("dropout", 0.25, 0.5)
    pos_weight_val = trial.suggest_float("pos_weight", 1, 1e4, log=True)
    gamma = trial.suggest_float("gamma", 0.75, 4.0)
    alpha = trial.suggest_float("alpha", 0.25, 1.0)
    decay = trial.suggest_float('decay', 0.6, 1.0)
    reg_str = trial.suggest_float("regularization_strength", 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)

    '''
    
    # ------------------------------ #
    #              DATA              #
    # ------------------------------ #
    aug_params = {
        "patch_size": (108,108,108),
        "final_size":   (108,108,108),
        "flip_prob":  0.5,
        "rot_prob":   0.5,
        "scale_prob": 1.0,
        "rot_range":  np.pi,
        "scale_range": 0.2
    }
    
    train_names, val_names = train_test_split(names, test_size=0.2)

    train_dataset = dataloader.MMapDataset(names=train_names, path=data_path, gpu=True, aug_params=aug_params, zero_threshold=zero_threshold)
    val_dataset = dataloader.MMapDataset(names=val_names, path=data_path, gpu=True, aug_params=aug_params)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=6, collate_fn=dataloader.custom_collate, shuffle=True, pin_memory=True) # can put data aug in collate func later to have optional aug
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=6, collate_fn=dataloader.custom_collate, shuffle=False, pin_memory=True)
    
    # ------------------------------ #
    #             MODEL              #
    # ------------------------------ #
    model = UNet(
        spatial_dims=3,
        in_channels=1,
        out_channels=1,
        channels=(64, 128, 256),
        strides=(3, 3),
        num_res_units=2,
        dropout=dropout,
        act = 'relu',
        bias=False
    ).to(device)
    
    # ------------------------------ #
    #        TRAINING METHODS        #
    # ------------------------------ #
    optimizer = Adam(model.parameters(), lr=lr)
    
    lr_scheduler = StepLR(optimizer, step_size=4, gamma=decay)
    pos_weight_scheduler = dataloader.Scheduler(warmup_epochs=warmup_epochs, warmup_value=1.0, init_value=pos_weight_val, factor=1.1, stop=20)
    gamma_scheduler = dataloader.Scheduler(warmup_epochs=warmup_epochs, warmup_value=1.0, init_value=gamma, factor=1.05, stop=10)

    # dice_loss = DiceLoss()
    def add_regularization_loss(model, regularization_strength):
        reg_loss = 0
        for param in model.parameters():
            reg_loss += torch.sum(param ** 2)
        return regularization_strength * reg_loss
    
    ''' Unused Loss Functions
    # dice_loss = DiceLoss(to_onehot_y=False, softmax=True, weight=weights).to(device)
    # focal_loss = FocalLoss(to_onehot_y=False, use_softmax=True, weight=weights, gamma=gamma ).to(device)
    # mse_loss = nn.MSELoss(reduction='none')
    '''
    
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=decay)
    i=0
    for epoch in range(num_epochs):
    # ------------------------------ #
    #             TRAIN              #
    # ------------------------------ #
        gamma = gamma_scheduler()
        pos_weight = pos_weight_scheduler()
        
        focal_loss = FocalLoss(gamma=gamma)
        bce_loss = BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight], device=device))
    
        model.train()
        batch_loss = 0
        opt_avg = 0
        tgt_avg = 0
        
        for batch in train_loader:
            input, target = batch['src'].to(device), batch['tgt'].to(device)
            optimizer.zero_grad()
            
            output = model(input)
            
            # output = torch.sigmoid(output)
            opt_avg += torch.mean(output)
            tgt_avg += torch.mean(target)
            # loss = 0.5 * focal_loss(output, target) + 0.5 * bce_loss(output,target)
            loss = (output.mean() - target.mean()).abs() + (output.var() - target.var()).abs()
            # loss = focal_loss(output,target)
            # reg_loss = add_regularization_loss(model, reg_str)
            # loss+=reg_loss

            loss.backward()
            optimizer.step()
            
            # print(f'epoch {epoch} batch {i} loss: {loss.item():.2f}')
            batch_loss+=loss.item()
        print(f'Epoch {i}')
        print(f'LR: {lr_scheduler.get_last_lr()[0]:.3e} Gamma: {gamma_scheduler():.4f} | Pos Weight: {pos_weight_scheduler():.4f}')
        print(f'Batch Loss: {(batch_loss / len(train_loader)):.4f} Pred Avg: {(opt_avg / len(train_loader)):.4f} | Tgt Avg: {(tgt_avg / len(train_loader)):.4f}')
        # print(f'Total Time: {total_time:.2f}s, Data Load Time: {(total_time - train_forward_time - train_backward_time):.2f}s, Forward Time: {train_forward_time:.2f}s, Backward Time: {train_backward_time:.2f}s')
        
        lr_scheduler.step()
        pos_weight_scheduler.step()
        gamma_scheduler.step()
        
        # ---------- #
        # VALIDATION #
        # ---------- #
        model.eval()
        val_loss = 0
        beta = 2.0  # Adjust for F1-score, for example, or set another value for different balance
        precision_metric = BinaryPrecision().to(device)
        recall_metric = BinaryRecall().to(device)
        sum_x = 0
        sum_x2 = 0
        count_x =0
        sum_y = 0
        sum_y2 = 0
        count_y =0
        with torch.no_grad():
            for batch in val_loader:
                input, target = batch['src'].to(device), batch['tgt'].to(device)
                output = model(input)
                # Threshold predictions to binary (e.g., 0.5)
                preds = output  # Apply sigmoid to get probabilities
                preds = (preds > 0.6).float()  # Threshold at 0.5

                # Threshold the target similarly
                target_bin = (target > 0.6).float()  # Convert target to binary based on the same threshold

                # Update metrics
                precision_metric.update(preds, target_bin)
                recall_metric.update(preds, target_bin)

                sum_x += output.sum()
                sum_x2 += (output ** 2).sum()
                count_x += output.numel()
                sum_y += target.sum()
                sum_y2 += (target ** 2).sum()
                count_y += target.numel()

            mean_x = sum_x / count_x
            var_x = (sum_x2 / count_x) - (mean_x ** 2)
            std_x = torch.sqrt(var_x)
            
            mean_y = sum_y / count_y
            var_y = (sum_y2 / count_y) - (mean_y ** 2)
            std_y = torch.sqrt(var_y)

            # Compute the final metrics
            precision = precision_metric.compute()
            recall = recall_metric.compute()
            fbeta = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
        if math.isnan(fbeta):
            fbeta = 0.0
        # Print out precision, recall, and F-beta
        print(f"Pred std / var: {std_x:.4f},{var_x:.4f}, Tgt std / var: {std_y:.4f},{var_y:.4f}")
        print(f"Validation Precision: {precision:.4f}, Recall: {recall:.4f}, F{beta}-score: {fbeta:.4f}")
        print()

        # Report to Optuna or any other hyperparameter optimization tool
        # val_loss /= len(val_loader)
        trial.report(fbeta, epoch)
        # print(f"Epoch {epoch} loss: {val_loss:.2f}")
            
        if trial.should_prune():
            
            raise optuna.exceptions.TrialPruned()
        i+=1

    return val_loss


study = optuna.create_study(direction="maximize", pruner=MedianPruner(n_startup_trials=3, n_warmup_steps=18))
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)

[I 2025-03-31 16:38:50,902] A new study created in memory with name: no-name-4b75cb66-9ee1-415c-9a92-289e6af693d8


Epoch 0
LR: 5.000e-04 Gamma: 1.0000 | Pos Weight: 1.0000
Batch Loss: 0.3614 Pred Avg: -0.1451 | Tgt Avg: 0.0009
Pred std / var: 0.1569,0.0246, Tgt std / var: 0.0285,0.0008
Validation Precision: 0.0020, Recall: 0.0028, F2.0-score: 0.0026

Epoch 1
LR: 5.000e-04 Gamma: 1.0500 | Pos Weight: 1.1000
Batch Loss: 0.1411 Pred Avg: -0.0669 | Tgt Avg: 0.0016
Pred std / var: 0.1069,0.0114, Tgt std / var: 0.0084,0.0001
Validation Precision: 0.0000, Recall: 0.0000, F2.0-score: 0.0000

Epoch 2
LR: 5.000e-04 Gamma: 1.1025 | Pos Weight: 1.2100
Batch Loss: 0.0727 Pred Avg: -0.0259 | Tgt Avg: 0.0017
Pred std / var: 0.0800,0.0064, Tgt std / var: 0.0161,0.0003
Validation Precision: 0.0000, Recall: 0.0000, F2.0-score: 0.0000

Epoch 3
LR: 5.000e-04 Gamma: 1.1576 | Pos Weight: 1.3310
Batch Loss: 0.0390 Pred Avg: 0.0013 | Tgt Avg: 0.0010
Pred std / var: 0.0680,0.0046, Tgt std / var: 0.0018,0.0000
Validation Precision: 0.0000, Recall: 0.0000, F2.0-score: 0.0000

Epoch 4
LR: 4.500e-04 Gamma: 1.2155 | Pos Weight:

[W 2025-03-31 16:53:34,257] Trial 0 failed with parameters: {'zero_threshold': 0.6715573237529597} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\hmhor\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\hmhor\AppData\Local\Temp\ipykernel_20364\2803042850.py", line 161, in objective
    for batch in val_loader:
  File "c:\Users\hmhor\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 630, in __next__
    data = self._next_data()
  File "c:\Users\hmhor\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1317, in _next_data
    self._shutdown_workers()
  File "c:\Users\hmhor\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1442, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK